In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import ElasticNet
from sklearn.metrics import  r2_score
import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb
import xgboost as xgb
from subprocess import check_output
print(check_output(['ls','input']).decode('utf8'))

sample_submission.csv
test.csv
train.csv



/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('input/train.csv', parse_dates=['start_date','creation_date','sell_date'])
test = pd.read_csv('input/test.csv', parse_dates=['start_date','creation_date','sell_date'])

print('Train data size: {} Test data size: {}'.format(train.shape, test.shape))

Train data size: (9366, 18) Test data size: (4801, 17)


In [3]:
train['desk_id'].fillna('DSK00000099', inplace=True)
test['desk_id'].fillna('DSK00000099', inplace=True)

train['desk_id']=train.desk_id.apply(lambda x : x[-4:]).astype(np.int16)
test['desk_id']=test.desk_id.apply(lambda x : x[-4:]).astype(np.int16)

In [4]:
le = LabelEncoder()
le.fit(train['office_id'])
train['office_id'] = le.transform(train['office_id'])
test['office_id'] = le.transform(test['office_id'])

In [5]:
le.fit(train['pf_category'])
train['pf_category'] = le.transform(train['pf_category'])
test['pf_category'] = le.transform(test['pf_category'])

In [6]:
train['start_creation'] = (train.start_date - train.creation_date).dt.days
test['start_creation'] = (test.start_date - test.creation_date).dt.days

In [7]:
train['start_sell'] = (train.start_date - train.sell_date).dt.days
test['start_sell'] = (test.start_date - test.sell_date).dt.days

In [8]:
train['creation_sell'] = (train.creation_date - train.sell_date).dt.days
test['creation_sell'] = (test.creation_date - test.sell_date).dt.days

In [9]:
train['curr_to_eur'] = train['currency'].apply(lambda x : 1.18 if x=='USD' else 1.17  if x=='CHF'
                                               else 133.28 if x=='JPY'  else 1)
test['curr_to_eur'] = test['currency'].apply(lambda x : 1.18 if x=='USD' else 1.17  if x=='CHF' 
                                                else 133.28 if x=='JPY'  else 1)

In [10]:
train['sold'].fillna(68552000.0, inplace= True)
train['bought'].fillna(68427270.0, inplace= True)

In [13]:
train['log_sold'] = np.log(train.sold.values)
test['log_sold'] = np.log(test.sold.values)

train['log_bought'] = np.log(train.bought.values)
test['log_bought'] = np.log(test.bought.values)

In [14]:
train['sold_in_eur'] = train['sold']*train['curr_to_eur']
test['sold_in_eur'] = test['sold']*test['curr_to_eur']

train['bought_in_eur'] = train['bought']*train['curr_to_eur']
test['bought_in_eur'] = test['bought']*test['curr_to_eur']

In [15]:
train['dff_sold_bought'] = (train.sold - train.bought)
test['dff_sold_bought'] = (test.sold - test.bought)

In [16]:
train['gain_loss'] = train.dff_sold_bought * train.euribor_rate *train['curr_to_eur']*train['start_sell']/30
test['gain_loss'] = test.dff_sold_bought * test.euribor_rate *test['curr_to_eur']*test['start_sell']/30

In [17]:
train.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)
test.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)

In [18]:
le.fit(train['country_code'])
train['country_code'] = le.transform(train['country_code'])
test['country_code'] = le.transform(test['country_code'])

In [19]:
le.fit(train['currency'])
train['currency'] = le.transform(train['currency'])
test['currency'] = le.transform(test['currency'])

In [20]:
train['libor_rate'].fillna(0.0, inplace= True)
test['libor_rate'].fillna(0.0, inplace=True)

In [21]:
train['sold_libor'] = train['sold']*train['libor_rate']
test['sold_libor'] = test['sold']*test['libor_rate']

train['sold_euribor'] = train['sold']*train['euribor_rate']
test['sold_euribor'] = test['sold']*test['euribor_rate']

In [22]:
train['indicator_code'].fillna(False, inplace= True)
test['indicator_code'].fillna(False, inplace= True)

le.fit(train['indicator_code'])
train['indicator_code'] = le.transform(train['indicator_code'])
test['indicator_code'] = le.transform(test['indicator_code'])

In [23]:
le.fit(train['type'])
train['type'] = le.transform(train['type'])
test['type'] = le.transform(test['type'])

In [24]:
train['hedge_value'].fillna(False, inplace=True)
test['hedge_value'].fillna(False, inplace=True)

train['hedge_value'] = train['hedge_value'].map({False:0, True:1})
test['hedge_value'] = test['hedge_value'].map({False:0, True:1})

In [25]:
train['status'].fillna(False, inplace= True)
test['status'].fillna(False, inplace= True)

train['status'] = train['status'].map({False:0, True:1})
train['status'] = train['status'].map({False:0, True:1})

In [26]:
retn = train['return']
test_portfolio_id = test.portfolio_id.values

#train.drop(['portfolio_id','bought', 'type', 'return'], inplace=True, axis=1)
#test.drop(['portfolio_id','bought', 'type'], inplace=True, axis=1)
train.drop(['portfolio_id',  'return'], inplace=True, axis=1)
test.drop(['portfolio_id'], inplace=True, axis=1)

In [27]:
params = {'learning_rate': 0.04, 
          'num_leaves':70,
          'min_data_in_leaf': 120,
          'max_depth': -1, 
          'colsample_bytree': 0.8,#0.522,
          'boosting': 'gbdt', 
          'application': 'regression', 
          "min_child_samples": 10,
          'metric': 'mae',  
          'verbosity': 0,
          'seed': 32}

In [28]:
lgb_train = lgb.Dataset(train, label=np.log1p(retn))

In [29]:
lgb_cv = lgb.cv(params, lgb_train, num_boost_round=10000, nfold= 5, stratified=False,
                 verbose_eval=100, early_stopping_rounds=40)

[100]	cv_agg's l1: 0.00151877 + 5.84868e-05
[200]	cv_agg's l1: 0.00131369 + 7.63759e-05
[300]	cv_agg's l1: 0.0012299 + 6.82696e-05
[400]	cv_agg's l1: 0.0011951 + 6.1853e-05
[500]	cv_agg's l1: 0.00117874 + 5.41754e-05
[600]	cv_agg's l1: 0.00117317 + 5.10973e-05
[700]	cv_agg's l1: 0.00116967 + 4.53581e-05


In [30]:
nround = lgb_cv['l1-mean'].index(np.min(lgb_cv['l1-mean'])) #mae
#nround = lgb_cv['rmse-mean'].index(np.min(lgb_cv['rmse-mean'])) #rmse
print(nround)

694


In [31]:
model = lgb.train(params, lgb_train, num_boost_round=nround)

In [32]:
r2_score(retn, np.expm1(model.predict(train)))

0.86728945983756212

In [37]:
lgb.plot_importance(model)

In [34]:
pred = model.predict(test)

In [35]:
sub = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':np.expm1(pred)})
sub.head(5)

portfolio_id    return
0   PF00001001  0.027623
1   PF00001004  0.033162
2   PF00001009  0.027874
3   PF00001013  0.024810
4   PF00001014  0.025147

In [36]:
sub.to_csv('lgb_v4.csv', index=False)